### neo4j 社区版 安装

https://hub.docker.com/layers/library/neo4j/4.4.32-community/images/sha256-ce25409b8c3cfaa9a63f4e182753d09266881893e667d0298935ad4bfb0f11e5?context=explore


In [ ]:
%%bash
docker pull neo4j:4.4.32-community
docker run -d -p 7474:7474 -p 7687:7687 --name neo4j -e "NEO4J_AUTH=neo4j/123456" neo4j:4.4.32-community

http://localhost:7474/browser/  ， 输入用户名、密码，  到达 neo4j 的web控制台


### neo4j基本例子

，来自 https://neo4j.com/docs/api/python-driver/5.18/#quick-example


In [5]:
from neo4j import GraphDatabase, RoutingControl


URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "123456")


def add_friend(driver, name, friend_name):
    driver.execute_query(
        "MERGE (a:Person {name: $name}) "
        "MERGE (friend:Person {name: $friend_name}) "
        "MERGE (a)-[:KNOWS]->(friend)",
        name=name, friend_name=friend_name, database_="neo4j",
    )


def print_friends(driver, name):
    records, _, _ = driver.execute_query(
        "MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
        "RETURN friend.name ORDER BY friend.name",
        name=name, database_="neo4j", routing_=RoutingControl.READ,
    )
    for record in records:
        print(record["friend.name"])


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    add_friend(driver, "Arthur", "Guinevere")
    add_friend(driver, "Arthur", "Lancelot")
    add_friend(driver, "Arthur", "Merlin")
    print_friends(driver, "Arthur")


Guinevere
Lancelot
Merlin


###   以pyvis可视化neo4j图，报错了，失败了


In [3]:
%%bash

pip install pyvis

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [6]:
from pyvis.network import Network

def visual(driver):
    _net = Network()
    records, _, _ = driver.execute_query(
        "MATCH (n)-[r]->(m)"
        "RETURN n, r, m LIMIT 100",
        database_="neo4j", routing_=RoutingControl.READ,
    )
    for record in records:
        srcV=record['n']['name']
        dstV=record['m']['name']
        _net.add_node(srcV)
        _net.add_node(dstV)
        _net.add_edge(srcV,dstV)
    return _net



driver= GraphDatabase.driver(URI, auth=AUTH)
net=visual(driver)
driver.close()

net.show("neo4j_graph.html")

neo4j_graph.html


AttributeError: 'NoneType' object has no attribute 'render'